# GNNS JSON-file Decoder

In [124]:

import numpy as np
from io import BytesIO
from scipy.interpolate import CubicSpline
import json


In [146]:
def get_coordinates_from_file(filename: str):
    '''Reads coordinates from a json file and returns them in there original structure'''
    with open(filename, 'r') as file:
        data = json.load(file)
    return data['coordinates']

def transform_from_nmea_coordinates_to_degrees(latitude_ddmm, longitude_ddmm) -> tuple[float, float]: 
    '''Converting the coordinates from nmea format (ddmm.mm) to degreees format (D.d)'''
    # Latitude conversion
    lat_degrees = int(latitude_ddmm // 100)  # Extracting degrees
    lat_minutes = latitude_ddmm % 100  # Extracting minutes
    latitude_decimal = lat_degrees + lat_minutes / 60  # Convert to decimal degrees

    # Longitude conversion
    lon_degrees = int(longitude_ddmm // 100)  # Extracting degrees
    lon_minutes = longitude_ddmm % 100  # Extracting minutes
    longitude_decimal = lon_degrees + lon_minutes / 60  # Convert to decimal degrees
    return latitude_decimal, longitude_decimal

def seperate_latitude_longitude_timestamps(coordinates: list):
    latitudes = []
    longitudes = []
    timestamps = []
    for c in coordinates:
        latitudes.append(c['lat'])
        longitudes.append(c['long'])
        timestamps.append(c['timeStamp'])
    return latitudes, longitudes, timestamps


def convert_timestamp_to_seconds(timestamp: str):
    # Convert 'hhmmss.mm' format to seconds
    hours = int(timestamp[:2])
    minutes = int(timestamp[2:4])
    seconds = int(timestamp[4:6])
    milliseconds = int(timestamp[7:])
    total_seconds = hours * 3600 + minutes * 60 + seconds + milliseconds / 1000
    return total_seconds


def interpolate_coordinates_linear(latitudes, longitudes, timestamps):
    # Convert strings to floats
    latitudes = [float(lat) for lat in latitudes]
    longitudes = [float(lon) for lon in longitudes]
    timestamps = [convert_timestamp_to_seconds(timestamp) for timestamp in timestamps]

    # Convert timestamps to milliseconds
    timestamps_ms = [timestamp * 1000 for timestamp in timestamps]

    # Create a linearly spaced array for each millisecond
    interpolated_timestamps = np.arange(timestamps_ms[0], timestamps_ms[-1] + 1, 1)

    # Interpolate latitude and longitude
    interpolated_latitudes = np.interp(interpolated_timestamps, timestamps_ms, latitudes)
    interpolated_longitudes = np.interp(interpolated_timestamps, timestamps_ms, longitudes)

    # Convert interpolated timestamps back to seconds
    interpolated_timestamps /= 1000

    return interpolated_latitudes, interpolated_longitudes, interpolated_timestamps


def interpolate_coordinates_smooth(latitudes, longitudes, timestamps):
    # Convert strings to floats
    latitudes = np.array([float(lat) for lat in latitudes])
    longitudes = np.array([float(lon) for lon in longitudes])
    timestamps = np.array([convert_timestamp_to_seconds(timestamp) for timestamp in timestamps])

    # Convert timestamps to milliseconds
    timestamps_ms = timestamps * 1000

    # Create a cubic spline interpolation for latitude and longitude
    spline_lat = CubicSpline(timestamps_ms, latitudes)
    spline_lon = CubicSpline(timestamps_ms, longitudes)

    # Interpolate at a higher resolution
    interpolated_timestamps_ms = np.arange(timestamps_ms[0], timestamps_ms[-1] + 1, 1)
    interpolated_latitudes = spline_lat(interpolated_timestamps_ms)
    interpolated_longitudes = spline_lon(interpolated_timestamps_ms)

    # Convert interpolated timestamps back to seconds
    interpolated_timestamps = interpolated_timestamps_ms / 1000

    return interpolated_latitudes, interpolated_longitudes, interpolated_timestamps


def get_position_at_time(latitudes:list, longitudes:list, timestamps:list, target_time:str):
    # Interpolate coordinates
    interpolated_latitudes, interpolated_longitudes, interpolated_timestamps = interpolate_coordinates_linear(
        latitudes, longitudes, timestamps)
    
    # Find the index of the target time in the interpolated timestamps
    target_time_seconds = convert_timestamp_to_seconds(target_time)
    index = np.searchsorted(interpolated_timestamps, target_time_seconds)
    print(index, len(interpolated_longitudes))

    # Return the corresponding latitude and longitude
    return interpolated_latitudes[index], interpolated_longitudes[index]

def write_smooth_geojson_file(input_filename: str, output_filename: str):
    coordinates = get_coordinates_from_file(input_filename)
    latitudes, longitudes, timestamps = seperate_latitude_longitude_timestamps(coordinates)
    interpolated_latitudes, interpolated_longitudes, interpolated_timestamps = interpolate_coordinates_smooth(latitudes, longitudes, timestamps)
    interpolated_coordinates = []
    for i in range(len(interpolated_timestamps)):
        i_lat = interpolated_latitudes[i]
        i_long = interpolated_longitudes[i]
        i_lat_dec, i_long_dec = transform_from_nmea_coordinates_to_degrees(i_lat, i_long)
        cord = [i_long_dec, i_lat_dec]
        interpolated_coordinates.append(cord)
    feature = {
        "type": "Feature",
        "properties": {},
        "geometry": {
            "type": "LineString",
            "coordinates": interpolated_coordinates
        }
    }
    with open(output_filename, 'w') as file:
        json.dump(feature, file)

def write_linear_geojson_file(input_filename: str, output_filename: str):
    coordinates = get_coordinates_from_file(input_filename)
    latitudes, longitudes, timestamps = seperate_latitude_longitude_timestamps(coordinates)
    interpolated_latitudes, interpolated_longitudes, interpolated_timestamps = interpolate_coordinates_linear(latitudes, longitudes, timestamps)
    interpolated_coordinates = []
    for i in range(len(interpolated_timestamps)):
        i_lat = interpolated_latitudes[i]
        i_long = interpolated_longitudes[i]
        i_lat_dec, i_long_dec = transform_from_nmea_coordinates_to_degrees(i_lat, i_long)
        cord = [i_long_dec, i_lat_dec]
        interpolated_coordinates.append(cord)
    feature = {
        "type": "Feature",
        "properties": {},
        "geometry": {
            "type": "LineString",
            "coordinates": interpolated_coordinates
        }
    }
    with open(output_filename, 'w') as file:
        json.dump(feature, file)


## Examples of usage

### Get coordinate from target time



In [147]:
coordinates = get_coordinates_from_file("recordings/buss.json") # recording from 105033.00 to 105859.75
latitudes, longitudes, timestamps = seperate_latitude_longitude_timestamps(coordinates)

target_time = "105342.61" # kl 10:53 42s 61ms
lat_at_time, long_at_time = get_position_at_time(latitudes, longitudes, timestamps, target_time)
print(f"NMEA format: lat - {lat_at_time}, long - {long_at_time}")


lat_at_time_in_deg, long_at_time_in_deg = transform_from_nmea_coordinates_to_degrees(lat_at_time, long_at_time)
print(f"Degrees format: lat - {lat_at_time_in_deg}, long - {long_at_time_in_deg}")



189061 506076
63.4217297158 10.400594608066667


### Write all interpolated coordinates to a GeoJSON file

In [134]:
write_linear_geojson_file("recordings/buss.json", "recordings/buss_o2.json")
